In [85]:
import pandas as pd

import torch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [101]:
df_parisHousing = pd.read_csv('src/ParisHousing.csv')

In [102]:
df_parisHousing.insert(0, 'Bias', 1)

In [103]:
X = df_parisHousing.drop(columns=['price'])
y = df_parisHousing.price

In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [105]:
scaler = StandardScaler()

feature_columns = X.columns[1:]
X_scaled_train = X_train.copy()
X_scaled_train[feature_columns] = scaler.fit_transform(X_train[feature_columns])

In [106]:
X_scaled_test = X_test.copy()
X_scaled_test[feature_columns] = scaler.transform(X_test[feature_columns])

In [107]:
X_train_t = torch.FloatTensor(X_scaled_train.values)
y_train_t = torch.FloatTensor(y_train.values)

In [114]:
X_test_t = torch.FloatTensor(X_scaled_test.values)
y_test_t = torch.FloatTensor(y_test.values)

In [11]:
def mse(y_pred, y_true):
    return torch.mean((y_pred - y_true) ** 2)

In [112]:
weights = torch.zeros(17, requires_grad=True)

num_iter = 1000

# Скорость обучения для параметров
learning_rate = 0.01

for i in range(num_iter):

    # Forward pass: предсказание модели
    y_pred_train = X_train_t @ weights

    # Подсчет оптимизируемой функции (MSE)
    loss = mse(y_pred_train, y_train_t)

    # Обратный проход: подсчет градиентов
    loss.backward()

    # Оптимизация: обновение параметров
    weights.data -= learning_rate * weights.grad.data

    # Зануление градиентов
    weights.grad.zero_()

    if (i + 1) % 100 == 0:
        print(f"Iter {i+1}, Loss: {loss.item():.4f}")

Iter 100, Loss: 607857213440.0000
Iter 200, Loss: 10786610176.0000
Iter 300, Loss: 196386368.0000
Iter 400, Loss: 7070501.5000
Iter 500, Loss: 3657253.5000
Iter 600, Loss: 3595464.2500
Iter 700, Loss: 3594323.7500
Iter 800, Loss: 3594393.2500
Iter 900, Loss: 3594400.5000
Iter 1000, Loss: 3594410.7500


In [119]:
from sklearn.metrics import r2_score

r2 = r2_score(y_train_t.numpy(), y_pred_train.detach().numpy())
r2

0.9999995827674866

In [116]:
y_pred_test = X_test_t @ weights

In [118]:
r2 = r2_score(y_test_t.numpy(), y_pred_test.detach().numpy())
r2

0.9999995827674866